In [ ]:
from ngsolve import *
from draw import Draw
from netgen.geom2d import unit_square

In [ ]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
fes = L2(mesh, order=2, dirichlet="bottom|right")
Draw(mesh)

In [ ]:
gfu = GridFunction(fes)  # solution 
u, v = fes.TnT()

In [ ]:
wind = CoefficientFunction((y-0.5,0.5-x))
#wind = 0.25*Normalize(wind)

In [ ]:
#u0 = exp (-40 * ( (x-0.75)*(x-0.75) + (y-0.5)*(y-0.5) ))
u0 = sqrt( (x-0.75)*(x-0.75) + (y-0.5)*(y-0.5) ) - 0.1

In [ ]:
n = specialcf.normal(mesh.dim)
wn = wind*n

c = BilinearForm(fes,nonassemble=True)
c += wind * grad(u) * v * dx + wn*(u.Other(bnd=u)-u)*IfPos(wn,0,v) * dx(element_boundary=True)

In [ ]:
gfu.Set(u0)
scene = Draw (gfu, mesh, "u", min=-0.1, max=0.7, autoscale=False)

In [ ]:
gfu.Set(u0)
from math import pi
t = 0
tau = 5e-4
tend = pi

F = fes.InvM() @ c.mat

cnt = 0
with TaskManager():
    while t < tend:
        gfu.vec.data -= tau * F * gfu.vec
        t += tau
        if cnt % 100 == 0:
            print("\r",t,end="")
            scene.Redraw()
        cnt += 1